In [1]:
import requests
import pandas as pd
# from datetime import datetime, timedelta, date
import datetime
import time
from polygon import RESTClient
import logging
import signal
import sys
import pickle
import lz4.frame  # type: ignore
import concurrent.futures
import os
import pandas as pd
import numpy as np
import glob
import nbimporter
import gzip

In [2]:
# Define the path to the saved .pickle.gz file
file_path = "C:\\Users\\SamuliMustonen\\Documents\\Ready Solutions\\Docs\\StockTrading\\Data\\rawComplete\\all_raw_data_2022-01-01_to_2024-09-30.pickle.gz"

# Load the DataFrame from the compressed pickle file
try:
    df = pd.read_pickle(file_path, compression='gzip')
    print("File loaded successfully!")
    print(df.head())  # Display the first few rows
except Exception as e:
    print(f"Error loading file: {e}")


File loaded successfully!
  symbol  close   open   high    low   volume   timestamp  \
0   AACT  10.12  10.10  10.15  10.10   603100  2023-06-12   
1   AACT  10.11  10.14  10.14  10.11     5500  2023-06-13   
2   AACT  10.10  10.11  10.13  10.10   143900  2023-06-14   
3   AACT  10.11  10.11  10.11  10.10  2061100  2023-06-15   
4   AACT  10.11  10.12  10.12  10.11   251000  2023-06-16   

   exceptionalCandleSize  ema20  ema50  openHigher  averageVolume  \
0                      0  10.12  10.12           0            NaN   
1                      0  10.12  10.12           0            NaN   
2                      0  10.12  10.12           0            NaN   
3                      0  10.12  10.12           0            NaN   
4                      0  10.12  10.12           0            NaN   

   strongVolume  strongVolume6MoPrior  accVolume  uptrend  
0             0                     0          0        0  
1             0                     0          0        0  
2           

In [3]:
# Check for missing values in the dataset
missing_data = df.isnull().sum()
print(missing_data)

symbol                        0
close                         0
open                          0
high                          0
low                           0
volume                        0
timestamp                     0
exceptionalCandleSize         0
ema20                         0
ema50                         0
openHigher                    0
averageVolume            113875
strongVolume                  0
strongVolume6MoPrior          0
accVolume                     0
uptrend                       0
dtype: int64


In [4]:
# Check for duplicates
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

Number of duplicate rows: 0


In [5]:
# Get summary statistics
print(df.describe())

               close           open           high            low  \
count  742709.000000  742709.000000  742709.000000  742709.000000   
mean       25.280034      25.294837      25.746279      24.836079   
std        44.232542      44.286322      44.986244      43.605948   
min         0.250000       0.250000       0.250000       0.130000   
25%         8.860000       8.870000       9.050000       8.670000   
50%        15.430000      15.440000      15.720000      15.160000   
75%        28.820000      28.830000      29.370000      28.290000   
max      1743.820000    1900.000000    2555.300000    1730.670000   

             volume  exceptionalCandleSize          ema20          ema50  \
count  7.427090e+05          742709.000000  742709.000000  742709.000000   
mean   8.446777e+05               0.034748      25.407491      25.616118   
std    2.944342e+06               0.183142      44.529832      45.078963   
min    0.000000e+00               0.000000       0.420000       0.600000  

In [6]:
# List all unique symbols in the 'symbol' column
unique_symbols = df['symbol'].unique()
print(unique_symbols)

['AACT' 'AAOI' 'AAT' ... 'ZKH' 'ZUO' 'ZYME']


In [8]:
# Filter for a specific symbol and date range within 2023
# symbol_filter = 'ABUS'  # Replace with the symbol you want to filter
date_filter = (df['timestamp'] >= '2023-01-01') & (df['timestamp'] <= '2023-12-31')
openHigher = (df['openHigher'] == 1)
accVolume = (df['accVolume']==1)
uptrend = (df['uptrend']==1)
excCandle = (df['exceptionalCandleSize']==1)

# Apply filters
filtered_df = df[date_filter & openHigher & accVolume & uptrend & excCandle]
# filtered_df = df[date_filter & openHigher & accVolume & uptrend & excCandle]

# View the filtered DataFrame
unique_symbols = filtered_df['symbol'].unique()
# unique_count_symbols = filtered_df['symbol'].nunique()
print(unique_symbols)
# print(filtered_df)

['AAOI' 'ACHR' 'ACMR' 'ADNT' 'ADSE' 'AGL' 'ALRS' 'AMBC' 'AMSC' 'AMWD'
 'ANIP' 'ANNX' 'APLD' 'APOG' 'ARDX' 'ARHS' 'ARLO' 'ARVN' 'ASPN' 'ASTL'
 'ATRC' 'ATRO' 'ATSG' 'AZTA' 'BASE' 'BBW' 'BELFA' 'BFST' 'BITF' 'BLBD'
 'BLND' 'BTBT' 'BTDR' 'BVS' 'BZH' 'CAL' 'CARS' 'CASH' 'CBRL' 'CDMO' 'CDRE'
 'CENTA' 'CERT' 'CGNT' 'CHCO' 'CIFR' 'CLOV' 'CNMD' 'CNTA' 'COCO' 'COUR'
 'CRESY' 'CRMD' 'CRSR' 'CSGS' 'CSIQ' 'CTOS' 'CVGW' 'CXM' 'DAKT' 'DAO'
 'DDL' 'DESP' 'DHC' 'DMRC' 'DOLE' 'ECVT' 'EDN' 'ENVX' 'EOSE' 'ETNB' 'ETWO'
 'EVCM' 'EVLV' 'EXAI' 'EYPT' 'FCF' 'FDP' 'FLGT' 'FLWS' 'FLYW' 'FNKO'
 'FSLY' 'FUBO' 'FVRR' 'FWRG' 'GBX' 'GCI' 'GCT' 'GDEN' 'GENI' 'GETY' 'GIC'
 'GIII' 'GLDD' 'GO' 'GOOS' 'GOTU' 'GPRE' 'HCI' 'HEPS' 'HLF' 'HLIT' 'HLX'
 'HSII' 'HTH' 'HUT' 'HY' 'IAS' 'IMNM' 'INOD' 'IPX' 'IREN' 'IRMD' 'IRON'
 'IRWD' 'ITRN' 'JACK' 'JBI' 'KC' 'KNSA' 'KRNT' 'KRUS' 'LAB' 'LASR' 'LGF.A'
 'LION' 'LMAT' 'LMB' 'LQDA' 'LRMR' 'LXRX' 'LZ' 'LZB' 'MDXG' 'MEG' 'MESO'
 'METC' 'METCB' 'ML' 'MLKN' 'MLR' 'MNKD' 'MNMD' 'MNTK' 'MRCY

In [12]:
# Filter for a specific symbol and date range within 2023
symbol_filter = 'AAOI'  # Replace with the symbol you want to filter
date_filter = (df['timestamp'] >= '2023-01-01') & (df['timestamp'] <= '2023-12-31')
openHigher = (df['openHigher'] == 1)
accVolume = (df['accVolume']==1)
uptrend = (df['uptrend']==1)
excCandle = (df['exceptionalCandleSize']==1)

# Apply all filters at once
filtered_df = df[
    (df['symbol'] == symbol_filter) & 
    date_filter & 
    openHigher & 
    accVolume & 
    uptrend & 
    excCandle
]

# View the filtered DataFrame
print(filtered_df)

    symbol  close   open   high    low    volume   timestamp  \
683   AAOI   4.85   4.70   4.99   4.55   3899100  2023-06-16   
686   AAOI   3.77   4.70   4.70   3.59   6429600  2023-06-22   
699   AAOI   9.54  10.32  10.38   8.88   6554000  2023-07-12   
716   AAOI  11.01   8.37  11.29   8.05  31770800  2023-08-04   
727   AAOI  15.44  13.49  15.59  13.36   7897900  2023-08-21   

     exceptionalCandleSize  ema20  ema50  openHigher  averageVolume  \
683                      1   3.00   2.51           1      309016.50   
686                      1   3.34   2.72           1      513565.16   
699                      1   6.11   4.36           1     1019798.03   
716                      1   7.60   6.16           1     1852912.69   
727                      1  11.58   8.76           1     2504015.37   

     strongVolume  strongVolume6MoPrior  accVolume  uptrend  
683             1                     1          1        1  
686             1                     1          1        1  
69